<a href="https://colab.research.google.com/github/nattntn/PMU-B-PersonalAI/blob/main/Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data form Kaggle

> TensorFlow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Basic
import os
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random
import numpy as np
import pandas as pd
from PIL import Image, ImageFile

# visuals
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
from PIL import Image

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay

# Tensorflow
import tensorflow as tf
from tensorflow.keras import callbacks
from keras.callbacks import Callback
from keras import layers
from keras import models
from tensorflow.keras import optimizers
from keras.optimizers import Adam
from efficientnet.keras import center_crop_and_resize, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#ดู version ของ tf
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
# dataset_url = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip' #url of datasets

In [ ]:
# !pip3 install tensorflow==2.14.0

In [ ]:
# !pip3 uninstall tensorflow
# !pip3 install tensorflow==2.14.0

In [ ]:
# print(tf.__version__)

**Upload API kaggle.json**

In [ ]:
! pip install -q kaggle

In [ ]:
#!mkdir -p /content/drive/MyDrive/Hackathon/.kaggle

In [ ]:
# !cp kaggle.json /content/drive/MyDrive/Hackathon/.kaggle/
# !chmod 600 /content/drive/MyDrive/Hackathon/.kaggle/kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
# !mkdir -p /content/drive/MyDrive/Hackathon/.kaggle
!cp kaggle.json ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
data_dir = '/content/drive/MyDrive/Hackathon'
%cd '/content/drive/MyDrive/Hackathon'
!pwd

/content/drive/MyDrive/Hackathon
/content/drive/MyDrive/Hackathon


In [ ]:
!kaggle datasets download -d shubhamgoel27/dermnet

100% 1.71G/1.72G [00:22<00:00, 105MB/s]
100% 1.72G/1.72G [00:22<00:00, 81.3MB/s]


In [ ]:
!unzip dermnet.zip

เอาต์พุตของการสตรีมมีการตัดเหลือเพียง 5000 บรรทัดสุดท้าย
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keloids-8.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keloids-80.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keloids-9.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-100.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-101.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-102.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-103.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-104.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-105.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-106.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoac

## Data Extraction

In [ ]:
import glob
# ดูจำนวนภาพใน folder
Cat_count = len(list(glob.glob(f'{data_dir}/PetImages/Cat/*.jpg'))) # Main folder:PetImages, sub forder:Cat,Dog
print('Cat: ', Cat_count)

Dog_count = len(list(glob.glob(f'{data_dir}/PetImages/Dog/*.jpg')))
print('Dog: ', Dog_count)

Cat:  12500
Dog:  12500


### Loading Images in a Dataframe

In [ ]:
df_dada = pd.DataFrame(columns=['filename', 'label']) # create dataframe
list_dir = ["Cat", "Dog"]
for label in list_dir:
    filenames = list(glob.glob(f'{data_dir}/PetImages/{label}/*.jpg'))
    #labels = [x.split("/")[3] for x in filenames]
    data = pd.DataFrame({"filename": filenames, "label": label})
    data = data.iloc[:1250,] #  1250 row / list_dir
    df_dada = df_dada.append(data)

print(df_dada.shape)
df_dada.head()

<ipython-input-14-9ce434094263>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_dada = df_dada.append(data)


(2500, 2)


<ipython-input-14-9ce434094263>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_dada = df_dada.append(data)


,filename,label
0,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Cat
1,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Cat
2,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Cat
3,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Cat
4,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Cat


In [ ]:
df_dada[df_dada['label']=='Cat']

,filename,label
0,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Cat
1,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Cat
2,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Cat
3,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Cat
4,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Cat
...,...,...
1245,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Cat
1246,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Cat
1247,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Cat
1248,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Cat


In [ ]:
df_dada[df_dada['label']=='Dog']

,filename,label
0,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Dog
1,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Dog
2,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Dog
3,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Dog
4,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Dog
...,...,...
1245,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Dog
1246,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Dog
1247,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Dog
1248,/content/drive/MyDrive/PMUB:Personal_AI/PetIma...,Dog


# Create dataframe

In [1]:
#เชื่อม google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## การทำ Path Filename

In [2]:
skin = r'/content/gdrive/MyDrive/Hackathon' # ให้อ่านที่อยู่ของโฟรเดอร์ในไดร์ฟ และสร้างตัวแปร ชื่อว่า paro7 มารับไว้

In [3]:
import os # os เอาไว้จัดการไฟล์และ path

#unwanted_folders = ['Detail pic.jpg', 'FILM NUMBER.gsheet'] # กำหนดชื่อโฟลเดอร์ที่ไม่ต้องการให้เก็บไฟล์

files = [] # สร้าง list เพื่อเก็บตำแหน่งของไฟล์
for path, dirnames, filenames in os.walk(skin): # os.walk คือ ทำงานสร้างชื่อไฟล์แต่ละไฟล์ ตามที่เก็บข้อมูล
   #if any(folder in path for folder in unwanted_folders): # ตรวจสอบว่า path เป็นโฟลเดอร์ที่ไม่ต้องการหรือไม่
       # continue # ข้ามไปยังโฟลเดอร์ถัดไป
    files.extend(os.path.join(path, name) for name in filenames) # เพิ่มตำแหน่งของไฟล์ลงใน list ถ้าเป็นไฟล์ในโฟลเดอร์ที่ต้องการ # extend  ใช้เพื่อรวม list # คำสั่งเชื่อมส่วนประกอบของ path ที่ประกอบด้วย directory หรือ file เพื่อสร้าง path หรือ URL ที่รองรับการทำงานบนระบบปฏิบัติการต่างๆ
print(files) # แสดงตำแหน่งของไฟล์ทั้งหมดที่อยู่ในโฟลเดอร์ที่ต้องการเก็บ

['/content/gdrive/MyDrive/Hackathon/dermnet.zip', '/content/gdrive/MyDrive/Hackathon/kaggle.json', '/content/gdrive/MyDrive/Hackathon/.kaggle/kaggle.json', '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/Dyshidrosis-47.jpg', '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/desquamation-4.jpg', '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/Dyshidrosis-3.jpg', '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/eczema-acute-4.jpg', '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/eczema-acute-5.jpg', '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/eczema-asteatotic-10.jpg', '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/Dyshidrosis-54.jpg', '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/03DermatitisLids1.jpg', '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/eczema-arms-6.jpg', '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/eczema-areola-5.jpg', '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/eczema-arms-10.jpg', '/content/g

In [4]:
files

['/content/gdrive/MyDrive/Hackathon/dermnet.zip',
 '/content/gdrive/MyDrive/Hackathon/kaggle.json',
 '/content/gdrive/MyDrive/Hackathon/.kaggle/kaggle.json',
 '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/Dyshidrosis-47.jpg',
 '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/desquamation-4.jpg',
 '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/Dyshidrosis-3.jpg',
 '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/eczema-acute-4.jpg',
 '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/eczema-acute-5.jpg',
 '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/eczema-asteatotic-10.jpg',
 '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/Dyshidrosis-54.jpg',
 '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/03DermatitisLids1.jpg',
 '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/eczema-arms-6.jpg',
 '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/eczema-areola-5.jpg',
 '/content/gdrive/MyDrive/Hackathon/test/Eczema Photos/eczema-arms-10.jpg'

In [5]:
len(files) #  pano images

4615

### ตาราง pathname

In [6]:
import pandas as pd # import ข้อมูลเข้าเพื่อสร้างตาาราง

In [7]:
pathname = pd.DataFrame({'Path_Name':files}) # สร้างตารางคอลลัมม์ที่อยู่ของภาพโดยตั้งชื่อคอลลัมม์ว่า Path_name
pathname #แสดงตาราง

,Path_Name
0,/content/gdrive/MyDrive/Hackathon/dermnet.zip
1,/content/gdrive/MyDrive/Hackathon/kaggle.json
2,/content/gdrive/MyDrive/Hackathon/.kaggle/kagg...
3,/content/gdrive/MyDrive/Hackathon/test/Eczema ...
4,/content/gdrive/MyDrive/Hackathon/test/Eczema ...
...,...
4610,/content/gdrive/MyDrive/Hackathon/train/Urtica...
4611,/content/gdrive/MyDrive/Hackathon/train/Urtica...
4612,/content/gdrive/MyDrive/Hackathon/train/Urtica...
4613,/content/gdrive/MyDrive/Hackathon/train/Urtica...


In [8]:
pathname = pathname.iloc[3:,:] # ตัดสามแถวบนออก
pathname

,Path_Name
3,/content/gdrive/MyDrive/Hackathon/test/Eczema ...
4,/content/gdrive/MyDrive/Hackathon/test/Eczema ...
5,/content/gdrive/MyDrive/Hackathon/test/Eczema ...
6,/content/gdrive/MyDrive/Hackathon/test/Eczema ...
7,/content/gdrive/MyDrive/Hackathon/test/Eczema ...
...,...
4610,/content/gdrive/MyDrive/Hackathon/train/Urtica...
4611,/content/gdrive/MyDrive/Hackathon/train/Urtica...
4612,/content/gdrive/MyDrive/Hackathon/train/Urtica...
4613,/content/gdrive/MyDrive/Hackathon/train/Urtica...


In [9]:
list_path = list(pathname['Path_Name']) # เก็บที่อยู่ของภาพในรูปแบบตาราง โดยชื่อว่า list_path ซึ่ง list(ชื่อตาราง['ชื่อคอลัมน์'])
for i in list_path: # วนลูปใน list_path
    res = i.split('/') # split ใช้สำหรับแยก โดยในที่นี้แยกด้วย / แล้วตั้งชื่อว่า res
    print(res) # แสดงผลลัพธ์ของ res ตัวที่ 7 คือตำเเหน่งที่เราต้องการ

['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', 'Dyshidrosis-47.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', 'desquamation-4.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', 'Dyshidrosis-3.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', 'eczema-acute-4.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', 'eczema-acute-5.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', 'eczema-asteatotic-10.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', 'Dyshidrosis-54.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', '03DermatitisLids1.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', 'eczema-arms-6.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', 'eczema-areola-5.jpg']
['', 'content', 'gdriv

In [10]:
list_path = list(pathname['Path_Name']) # เก็บที่อยู่ของภาพในรูปแบบตาราง โดยชื่อว่า list_path ซึ่ง list(ชื่อตาราง['ชื่อคอลัมน์'])
class_type = list() # สร้างตัวแปร มาเก็บ list ว่าง
class_disease = list()
class_ima = list()
for i in list_path: # วนลูปใน list_path
    res = i.split('/') # split ใช้สำหรับแยก โดยในที่นี้แยกด้วย / แล้วตั้งชื่อว่า res
    print(res) # แสดงผลลัพธ์ของ res ตัวที่ 7 คือตำเเหน่งที่เราต้องการ
    class_type.append(res[-3]) # append  คำสั่งเพิ่มข้อมูลลงใน List
    class_disease.append(res[-2]) # append  คำสั่งเพิ่มข้อมูลลงใน List
    class_ima.append(res[-1]) # append  คำสั่งเพิ่มข้อมูลลงใน List

pathname['Type'] = class_type # ตั้งชื่อคอลัมน์ว่า Type ในตาราง pathname โดยนำค่าใน class_type มาใส่
pathname['Disease'] = class_disease
pathname['Filename'] = class_ima

['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', 'Dyshidrosis-47.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', 'desquamation-4.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', 'Dyshidrosis-3.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', 'eczema-acute-4.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', 'eczema-acute-5.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', 'eczema-asteatotic-10.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', 'Dyshidrosis-54.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', '03DermatitisLids1.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', 'eczema-arms-6.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'test', 'Eczema Photos', 'eczema-areola-5.jpg']
['', 'content', 'gdriv

<ipython-input-10-d755ca00b813>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pathname['Type'] = class_type # ตั้งชื่อคอลัมน์ว่า Type ในตาราง pathname โดยนำค่าใน class_type มาใส่
<ipython-input-10-d755ca00b813>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pathname['Disease'] = class_disease
<ipython-input-10-d755ca00b813>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [11]:
pathname

,Path_Name,Type,Disease,Filename
3,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,Dyshidrosis-47.jpg
4,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,desquamation-4.jpg
5,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,Dyshidrosis-3.jpg
6,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,eczema-acute-4.jpg
7,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,eczema-acute-5.jpg
...,...,...,...,...
4610,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-7.jpg
4611,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-13.jpg
4612,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-88.jpg
4613,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-101.jpg


In [12]:
set(pathname['Disease'])

{'Eczema Photos',
 'Herpes HPV and other STDs Photos',
 'Psoriasis pictures Lichen Planus and related diseases',
 'Scabies Lyme Disease and other Infestations and Bites',
 'Urticaria Hives'}

In [36]:
table = pathname.reset_index()
table

,index,Path_Name,Type,Disease,Filename
0,3,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,Dyshidrosis-47.jpg
1,4,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,desquamation-4.jpg
2,5,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,Dyshidrosis-3.jpg
3,6,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,eczema-acute-4.jpg
4,7,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,eczema-acute-5.jpg
...,...,...,...,...,...
4607,4610,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-7.jpg
4608,4611,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-13.jpg
4609,4612,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-88.jpg
4610,4613,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-101.jpg


In [37]:
table = table.iloc[:,1:]
table

,Path_Name,Type,Disease,Filename
0,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,Dyshidrosis-47.jpg
1,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,desquamation-4.jpg
2,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,Dyshidrosis-3.jpg
3,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,eczema-acute-4.jpg
4,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,eczema-acute-5.jpg
...,...,...,...,...
4607,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-7.jpg
4608,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-13.jpg
4609,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-88.jpg
4610,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-101.jpg


In [22]:
# table.to_csv('/content/gdrive/MyDrive/Hackathon/Table_all.csv')

# Data split

In [38]:
import pandas as pd

# Assuming your DataFrame is named df
# Replace df with the actual name of your DataFrame if different

# Create a dictionary to map disease names to category names
disease_to_category = {
    'Scabies Lyme Disease and other Infestations and Bites': 'Cat1',
    'Urticaria Hives': 'Cat1',
    'Herpes HPV and other STDs Photos': 'Cat2',
    'Eczema Photos': 'Cat2',
    'Atopic Dermatitis Photos': 'Cat2',
    'Psoriasis pictures Lichen Planus and related diseases': 'Cat2',
    # Add more mappings as needed
}

# Map the values in the "Disease" column to the corresponding categories
table['Category'] = table['Disease'].map(disease_to_category)

# Display the updated DataFrame
print(table)


                                              Path_Name   Type  \
0     /content/gdrive/MyDrive/Hackathon/test/Eczema ...   test   
1     /content/gdrive/MyDrive/Hackathon/test/Eczema ...   test   
2     /content/gdrive/MyDrive/Hackathon/test/Eczema ...   test   
3     /content/gdrive/MyDrive/Hackathon/test/Eczema ...   test   
4     /content/gdrive/MyDrive/Hackathon/test/Eczema ...   test   
...                                                 ...    ...   
4607  /content/gdrive/MyDrive/Hackathon/train/Urtica...  train   
4608  /content/gdrive/MyDrive/Hackathon/train/Urtica...  train   
4609  /content/gdrive/MyDrive/Hackathon/train/Urtica...  train   
4610  /content/gdrive/MyDrive/Hackathon/train/Urtica...  train   
4611  /content/gdrive/MyDrive/Hackathon/train/Urtica...  train   

              Disease                       Filename Category  
0       Eczema Photos             Dyshidrosis-47.jpg     Cat2  
1       Eczema Photos             desquamation-4.jpg     Cat2  
2       Eczema 

In [39]:
table

,Path_Name,Type,Disease,Filename,Category
0,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,Dyshidrosis-47.jpg,Cat2
1,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,desquamation-4.jpg,Cat2
2,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,Dyshidrosis-3.jpg,Cat2
3,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,eczema-acute-4.jpg,Cat2
4,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,eczema-acute-5.jpg,Cat2
...,...,...,...,...,...
4607,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-7.jpg,Cat1
4608,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-13.jpg,Cat1
4609,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-88.jpg,Cat1
4610,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-101.jpg,Cat1


In [72]:
print(len(table[table['Disease']=='Eczema Photos']))
print(len(table[table['Disease']=='Herpes HPV and other STDs Photos']))
print(len(table[table['Disease']=='Psoriasis pictures Lichen Planus and related diseases']))
print(len(table[table['Disease']=='Scabies Lyme Disease and other Infestations and Bites']))
print(len(table[table['Disease']=='Urticaria Hives']))

1544
507
1757
539
265


In [73]:
print(len(table[table['Category']=='Cat1']))
print(len(table[table['Category']=='Cat2']))

804
3808


In [40]:
# table.to_csv('/content/gdrive/MyDrive/Hackathon/Table_all_Catagory.csv')

## Test

In [41]:
table_test = table[table['Type']== 'test']
table_test

,Path_Name,Type,Disease,Filename,Category
0,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,Dyshidrosis-47.jpg,Cat2
1,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,desquamation-4.jpg,Cat2
2,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,Dyshidrosis-3.jpg,Cat2
3,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,eczema-acute-4.jpg,Cat2
4,/content/gdrive/MyDrive/Hackathon/test/Eczema ...,test,Eczema Photos,eczema-acute-5.jpg,Cat2
...,...,...,...,...,...
919,/content/gdrive/MyDrive/Hackathon/test/Urticar...,test,Urticaria Hives,PUPPP-26.jpg,Cat1
920,/content/gdrive/MyDrive/Hackathon/test/Urticar...,test,Urticaria Hives,dermagraphism-20.jpg,Cat1
921,/content/gdrive/MyDrive/Hackathon/test/Urticar...,test,Urticaria Hives,erythema-annulare-centrifugum-9.jpg,Cat1
922,/content/gdrive/MyDrive/Hackathon/test/Urticar...,test,Urticaria Hives,dermagraphism-14.jpg,Cat1


In [65]:
print(len(table_test[table_test['Disease']=='Eczema Photos']))
print(len(table_test[table_test['Disease']=='Herpes HPV and other STDs Photos']))
print(len(table_test[table_test['Disease']=='Psoriasis pictures Lichen Planus and related diseases']))
print(len(table_test[table_test['Disease']=='Scabies Lyme Disease and other Infestations and Bites']))
print(len(table_test[table_test['Disease']=='Urticaria Hives']))

309
102
352
108
53


In [42]:
# table_test.to_csv('/content/gdrive/MyDrive/Hackathon/Table_test.csv')

## Validation

In [44]:
table_train_val = table[table['Type']== 'train'].reset_index()
table_train_val

,index,Path_Name,Type,Disease,Filename,Category
0,924,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-chronic-51.jpg,Cat2
1,925,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-foot-11.jpg,Cat2
2,926,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-fingertips-106.jpg,Cat2
3,927,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-face-6.jpg,Cat2
4,928,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-asteatotic-8.jpg,Cat2
...,...,...,...,...,...,...
3683,4607,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-7.jpg,Cat1
3684,4608,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-13.jpg,Cat1
3685,4609,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-88.jpg,Cat1
3686,4610,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-101.jpg,Cat1


In [45]:
table_train_val = table_train_val.iloc[:,1:]
table_train_val

,Path_Name,Type,Disease,Filename,Category
0,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-chronic-51.jpg,Cat2
1,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-foot-11.jpg,Cat2
2,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-fingertips-106.jpg,Cat2
3,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-face-6.jpg,Cat2
4,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-asteatotic-8.jpg,Cat2
...,...,...,...,...,...
3683,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-7.jpg,Cat1
3684,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-13.jpg,Cat1
3685,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-88.jpg,Cat1
3686,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-101.jpg,Cat1


In [46]:
import numpy as np
import pandas as pd

np.random.seed(99)
# สร้าง DataFrame ที่มีจำนวนแถวเท่ากับ 3688
shuffled_df = pd.DataFrame()

# กำหนดจำนวนแถวที่ต้องการ (ในที่นี้คือ 3688)
num_rows = 3688

# สร้างคอลัมน์ Fold และใส่ค่าสุ่มในช่วง 1-10 ให้กับแต่ละแถว
shuffled_df['Fold'] = np.random.randint(1, 6, num_rows)

# แสดง DataFrame ที่ได้
print(shuffled_df)


      Fold
0        2
1        4
2        2
3        1
4        2
...    ...
3683     4
3684     4
3685     5
3686     3
3687     3

[3688 rows x 1 columns]


In [47]:
shuffled_df

,Fold
0,2
1,4
2,2
3,1
4,2
...,...
3683,4
3684,4
3685,5
3686,3


In [48]:
# เพิ่มคอลัมน์ Fold จาก DataFrame ใหม่เข้าไปใน DataFrame เดิม
existing_df = table_train_val.assign(Fold=shuffled_df['Fold'])
existing_df

,Path_Name,Type,Disease,Filename,Category,Fold
0,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-chronic-51.jpg,Cat2,2
1,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-foot-11.jpg,Cat2,4
2,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-fingertips-106.jpg,Cat2,2
3,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-face-6.jpg,Cat2,1
4,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-asteatotic-8.jpg,Cat2,2
...,...,...,...,...,...,...
3683,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-7.jpg,Cat1,4
3684,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-13.jpg,Cat1,4
3685,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-88.jpg,Cat1,5
3686,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-101.jpg,Cat1,3


In [60]:
set(existing_df['Disease'])

{'Eczema Photos',
 'Herpes HPV and other STDs Photos',
 'Psoriasis pictures Lichen Planus and related diseases',
 'Scabies Lyme Disease and other Infestations and Bites',
 'Urticaria Hives'}

In [62]:
print(len(existing_df[(existing_df['Fold']==1)&(existing_df['Disease']=='Eczema Photos')]))
print(len(existing_df[(existing_df['Fold']==1)&(existing_df['Disease']=='Herpes HPV and other STDs Photos')]))
print(len(existing_df[(existing_df['Fold']==1)&(existing_df['Disease']=='Psoriasis pictures Lichen Planus and related diseases')]))
print(len(existing_df[(existing_df['Fold']==1)&(existing_df['Disease']=='Scabies Lyme Disease and other Infestations and Bites')]))
print(len(existing_df[(existing_df['Fold']==1)&(existing_df['Disease']=='Urticaria Hives')]))

233
63
271
95
46


In [63]:
print(len(existing_df[(existing_df['Fold']==2)&(existing_df['Disease']=='Eczema Photos')]))
print(len(existing_df[(existing_df['Fold']==2)&(existing_df['Disease']=='Herpes HPV and other STDs Photos')]))
print(len(existing_df[(existing_df['Fold']==2)&(existing_df['Disease']=='Psoriasis pictures Lichen Planus and related diseases')]))
print(len(existing_df[(existing_df['Fold']==2)&(existing_df['Disease']=='Scabies Lyme Disease and other Infestations and Bites')]))
print(len(existing_df[(existing_df['Fold']==2)&(existing_df['Disease']=='Urticaria Hives')]))

237
87
288
74
45


In [64]:
print(len(existing_df[(existing_df['Fold']==3)&(existing_df['Disease']=='Eczema Photos')]))
print(len(existing_df[(existing_df['Fold']==3)&(existing_df['Disease']=='Herpes HPV and other STDs Photos')]))
print(len(existing_df[(existing_df['Fold']==3)&(existing_df['Disease']=='Psoriasis pictures Lichen Planus and related diseases')]))
print(len(existing_df[(existing_df['Fold']==3)&(existing_df['Disease']=='Scabies Lyme Disease and other Infestations and Bites')]))
print(len(existing_df[(existing_df['Fold']==3)&(existing_df['Disease']=='Urticaria Hives')]))

242
81
269
94
39


In [70]:
print(len(existing_df[(existing_df['Fold']==4)&(existing_df['Disease']=='Eczema Photos')]))
print(len(existing_df[(existing_df['Fold']==4)&(existing_df['Disease']=='Herpes HPV and other STDs Photos')]))
print(len(existing_df[(existing_df['Fold']==4)&(existing_df['Disease']=='Psoriasis pictures Lichen Planus and related diseases')]))
print(len(existing_df[(existing_df['Fold']==4)&(existing_df['Disease']=='Scabies Lyme Disease and other Infestations and Bites')]))
print(len(existing_df[(existing_df['Fold']==4)&(existing_df['Disease']=='Urticaria Hives')]))

273
83
301
83
41


In [71]:
print(len(existing_df[(existing_df['Fold']==5)&(existing_df['Disease']=='Eczema Photos')]))
print(len(existing_df[(existing_df['Fold']==5)&(existing_df['Disease']=='Herpes HPV and other STDs Photos')]))
print(len(existing_df[(existing_df['Fold']==5)&(existing_df['Disease']=='Psoriasis pictures Lichen Planus and related diseases')]))
print(len(existing_df[(existing_df['Fold']==5)&(existing_df['Disease']=='Scabies Lyme Disease and other Infestations and Bites')]))
print(len(existing_df[(existing_df['Fold']==5)&(existing_df['Disease']=='Urticaria Hives')]))

250
91
276
85
41


In [78]:
Dis1 = existing_df[(existing_df['Fold']==2)&(existing_df['Disease']=='Eczema Photos')].iloc[:128,:]
Dis2 = existing_df[(existing_df['Fold']==2)&(existing_df['Disease']=='Herpes HPV and other STDs Photos')]
Dis3 = existing_df[(existing_df['Fold']==2)&(existing_df['Disease']=='Psoriasis pictures Lichen Planus and related diseases')].iloc[:128,:]
Dis4 = existing_df[(existing_df['Fold']==2)&(existing_df['Disease']=='Scabies Lyme Disease and other Infestations and Bites')]
Dis5 = existing_df[(existing_df['Fold']==2)&(existing_df['Disease']=='Urticaria Hives')]

In [79]:
table_val = pd.concat([Dis1,Dis2,Dis3,Dis4,Dis5])
table_val

,Path_Name,Type,Disease,Filename,Category,Fold
0,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-chronic-51.jpg,Cat2,2
2,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-fingertips-106.jpg,Cat2,2
4,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-asteatotic-8.jpg,Cat2,2
9,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-chronic-21.jpg,Cat2,2
11,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-asteatotic-49.jpg,Cat2,2
...,...,...,...,...,...,...
3670,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-80.jpg,Cat1,2
3671,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,pressure-urticaria-7.jpg,Cat1,2
3675,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-98.jpg,Cat1,2
3676,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-62.jpg,Cat1,2


In [89]:
print(len(table_val[table_val['Category']=='Cat1']))
print(len(table_val[table_val['Category']=='Cat2']))

119
343


In [90]:
table_val = table_val.reset_index()
table_val

,index,Path_Name,Type,Disease,Filename,Category,Fold
0,0,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-chronic-51.jpg,Cat2,2
1,2,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-fingertips-106.jpg,Cat2,2
2,4,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-asteatotic-8.jpg,Cat2,2
3,9,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-chronic-21.jpg,Cat2,2
4,11,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-asteatotic-49.jpg,Cat2,2
...,...,...,...,...,...,...,...
457,3670,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-80.jpg,Cat1,2
458,3671,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,pressure-urticaria-7.jpg,Cat1,2
459,3675,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-98.jpg,Cat1,2
460,3676,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-62.jpg,Cat1,2


In [91]:
table_val = table_val.iloc[:,1:]
table_val

,Path_Name,Type,Disease,Filename,Category,Fold
0,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-chronic-51.jpg,Cat2,2
1,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-fingertips-106.jpg,Cat2,2
2,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-asteatotic-8.jpg,Cat2,2
3,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-chronic-21.jpg,Cat2,2
4,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-asteatotic-49.jpg,Cat2,2
...,...,...,...,...,...,...
457,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-80.jpg,Cat1,2
458,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,pressure-urticaria-7.jpg,Cat1,2
459,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-98.jpg,Cat1,2
460,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-62.jpg,Cat1,2


In [92]:
# table_val.to_csv(' ')

## Train

In [93]:
train = existing_df[~existing_df['Filename'].isin(table_val['Filename'])]
train

,Path_Name,Type,Disease,Filename,Category,Fold
1,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-foot-11.jpg,Cat2,4
3,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-face-6.jpg,Cat2,1
5,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-foot-74.jpg,Cat2,1
6,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-foot-9.jpg,Cat2,3
7,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-fingertips-94.jpg,Cat2,5
...,...,...,...,...,...,...
3683,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-7.jpg,Cat1,4
3684,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-13.jpg,Cat1,4
3685,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-88.jpg,Cat1,5
3686,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-101.jpg,Cat1,3


In [94]:
print(len(train[train['Category']=='Cat1']))
print(len(train[train['Category']=='Cat2']))

524
2702


In [95]:
table_train = train.reset_index()
table_train

,index,Path_Name,Type,Disease,Filename,Category,Fold
0,1,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-foot-11.jpg,Cat2,4
1,3,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-face-6.jpg,Cat2,1
2,5,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-foot-74.jpg,Cat2,1
3,6,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-foot-9.jpg,Cat2,3
4,7,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-fingertips-94.jpg,Cat2,5
...,...,...,...,...,...,...,...
3221,3683,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-7.jpg,Cat1,4
3222,3684,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-13.jpg,Cat1,4
3223,3685,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-88.jpg,Cat1,5
3224,3686,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-101.jpg,Cat1,3


In [96]:
table_train = table_train.iloc[:,1:]
table_train

,Path_Name,Type,Disease,Filename,Category,Fold
0,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-foot-11.jpg,Cat2,4
1,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-face-6.jpg,Cat2,1
2,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-foot-74.jpg,Cat2,1
3,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-foot-9.jpg,Cat2,3
4,/content/gdrive/MyDrive/Hackathon/train/Eczema...,train,Eczema Photos,eczema-fingertips-94.jpg,Cat2,5
...,...,...,...,...,...,...
3221,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-7.jpg,Cat1,4
3222,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-13.jpg,Cat1,4
3223,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-88.jpg,Cat1,5
3224,/content/gdrive/MyDrive/Hackathon/train/Urtica...,train,Urticaria Hives,hives-Urticaria-Acute-101.jpg,Cat1,3


In [98]:
# table_train.to_csv('/content/gdrive/MyDrive/Hackathon/Table_train.csv')

# Effnet

# Install

In [ ]:
!pip install efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.9 MB/s eta 0:00:00


# **Loading Library**

In [ ]:
!pip install efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Basic
import os
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random
import numpy as np
import pandas as pd
from PIL import Image, ImageFile

# visuals
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
from PIL import Image

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay

# Tensorflow
import tensorflow as tf
from tensorflow.keras import callbacks
from keras.callbacks import Callback
from keras import layers
from keras import models
from tensorflow.keras import optimizers
from keras.optimizers import Adam
from efficientnet.keras import center_crop_and_resize, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
!pwd

/content


In [ ]:
%cd '/content/train/Scabies Lyme Disease and other Infestations and Bites'

/content/train/Scabies Lyme Disease and other Infestations and Bites
